# 関数呼び出し機能(function calling)の利用

2023年の6月にOpeAI社ｎAPIに追加された[関数呼び出し機能](https://openai.com/blog/function-calling-and-other-api-updates)を利用することで、入力内容から情報を抽出・変換することができます。  
実際に呼び出す処理はユーザー側で用意する必要があり、APIで行われるのは関数を実行しやすいように整形する処理となります。  
関数の内容はfunction callの例に合わせています。  

現在はOpenAI系のAPIのみの機能ですが、他のLLMでもOpenAIのAPIとの互換性は重視されており、今後利用可能になることが予想されます。  
モデルのバージョンは`0613`以降でしか利用はできません。またファインチューニングしたモデルでは2023年8月現在では利用不可となっていますが将来的には利用可能になるそうです。  

In [ ]:
* API Key ;
%let api_key = OpenAI API key ;

## 自然言語からの情報抽出・変換

関数を呼び出さなくても、関数の形で必要な情報を定義することで、json形式の出力で情報の抽出や変換を行うことができます。  
以下の例では、`今の東京の天気は？`という質問を`{"location": "Tokyo"}`というようなJsonの形式に抽出・変換することができます。  
適切な関数がない場合は、通常と同じ動作となります。  

インデントは動作に影響はしないので、strip関数等でインデントを削除すればトークン数が節約できます。
回答が格納される階層が通常の場合と異なるので注意してください。

In [2]:
filename response temp ;
filename payload temp ;

%let query = 今の東京の天気は? ;

data _null_ ;
  infile cards ;
  file payload ;
  if _n_ = 1 then put 
    %unquote(%bquote('{"model":"gpt-3.5-turbo", "temperature":0, "messages" : [{"role": "user", "content": "&query."}], ')) ; 

  input ;
  put _infile_ ;
cards ;
"functions" : [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required": ["location"]
        }
    }
]
}
;
run ;

proc http url = "https://api.openai.com/v1/chat/completions"
          method = "POST"
          in = payload
          out = response
          ;
          headers "Content-type" = "application/json"
                  "Accept" = "application/json"
                  "Authorization" = "Bearer &api_key."
                ;
run ;
 
libname content json fileref=response ;

data work.param ;
  set content.MESSAGE_FUNCTION_CALL ;
run ;

proc print data = work.param ;
run ;

OBS,ordinal_message,ordinal_function_call,name,arguments
1,1,1,get_current_weather,"{ ""location"": ""Tokyo"" }"


## 動的なマクロの実行
function callでは関数を複数設定が可能で、関数とそれにあわせたパラメータが入力内容から選択・抽出できます。  
SAS上では呼び出し先はマクロとして、中間処理用のマクロを用意しておくことで動的に既存のマクロを実行します。  

以下ではサンプルとして今の天気と、現在の時間を取得するような関数の2つを定義しています。

In [ ]:
filename response temp ;
filename payload temp ;

%let query = 今の東京の時間は? ;

data _null_ ;
  infile cards ;
  file payload ;
  if _n_ = 1 then put 
    %unquote(%bquote('{"model":"gpt-3.5-turbo", "temperature":0, "messages" : [{"role": "user", "content": "&query."}], ')) ; 

  input ;
  put _infile_ ;
cards ;
"functions" : [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_datetime",
        "description": "Get the current datetime in iso8601 format",
        "parameters": {"type": "object", "properties": {}}
    }
]
}
;
run ;

proc http url = "https://api.openai.com/v1/chat/completions"
          method = "POST"
          in = payload
          out = response
          ;
          headers "Content-type" = "application/json"
                  "Accept" = "application/json"
                  "Authorization" = "Bearer &api_key."
                ;
run ;

/* sample macro to be called */
%macro get_current_weather(location=, unit=) ;
  data _null_ ;
    file print ;
    put "&sysmacroname. location:&location. unit:&unit." ;
  run ;
%mend ;

%macro get_datetime ;
  data _null_ ;
    file print ;
    datetime = put(datetime(), e8601dt.) ;
    put "&sysmacroname. " datetime ;
  run ;
%mend ;

既存のプログラムを動的に実行させるために、ここでは中間処理用のマクロで各種情報を取得した後、最終的なマクロを実行します。  

以下のマクロでは一度libname jsonでデータセット化した後、ファイルに出力して再度jsonとして読み直し、変数名と値をパラメータとして設定してマクロを実行します。

`今の東京の時間は?`という入力内容にあったマクロが実行されていることが確認できます。

In [8]:
libname content json fileref=response ;

%macro function_call2 ;
  filename args temp ;
  data _null_ ;
    file args ;
    set content.message_function_call ;
    put arguments ;
    call symputx('name', name) ;
    /* if parameter is None */
    if arguments = "{}" then do ;
      call execute(cats('%',"&name")) ;
      call symputx("_nonefl", 1) ;
    end ;
    else call symputx("_nonefl", 0) ;
  run ;
  
  %if &_nonefl. = 1 %then %goto exit ;

  libname param json fileref=args ;

  /* get parameter type * */
  data _null_ ;
    set sashelp.vcolumn end = eof ;
    where libname = "PARAM" and memname = "ROOT" and name ^= "ordinal_root" ;
    retain _numfl _charfl 0 ;
    if type = "num" then _numfl = 1 ;
    else if type = "char" then _charfl = 1 ;
    if eof = 1 then do ;  
      call symputx("numfl", _numfl) ;
      call symputx("charfl", _charfl) ;
    end ;
  run ;
  
  data _null_ ;
    set param.root(drop=ordinal_root) ;
    length _param $1000 ;
    
    %if &numfl. = 1 %then %do ;
    array _num _numeric_ ;
    do over _num ;
      _param = catx(",", _param, catx("=",vname(_num), _num)) ;
    end ;
    %end ;
    %if &charfl. = 1 %then %do ;
    array _char _character_ ;
    do over _char ;
      if vname(_char) ^= "_param" then continue ;
      _param = catx(",", _param, catx("=",vname(_char), _char)) ;
    end ;
    %end ;
    call execute(cats('%',"&name",'(',_param,')')) ;
  run ;

  %exit:
%mend ;

%function_call2 ;

以下にはじめに作成したテキスト処理によるマクロも一応置いておきます。  
内容によりエスケープ処理が必要になることが考えられるので注意してください。

In [9]:

%macro function_call(name=, arguments=) ;
  %if &arguments. = {} %then %do ;
    %&name. ;
  %end ;
  %else %do ;
    %let param = ;
    %let n_key = %eval(%sysfunc(countc(&arguments., %str(,)))+1) ;  
    %do i = 1 %to %eval(&n_key.) ;
      %let item =  %scan(&arguments., &i., %str(,)) ;
      %let key = %sysfunc(prxchange(s/^.*"(.+)"\s*:\s*([^,\}]+).*/$1/, -1, &item.)) ;
      %let value = %sysfunc(dequote(%sysfunc(prxchange(s/^.*"(.+)"\s*:\s*([^,\}]+).*/$2/, -1, &item.)))) ;
      %let param = &param. &key. = &value. ;

      %if &i. ^= &n_key. %then %let param = &param. , ;
    %end ;
    %&name.(&param.) ;
  %end ;
%mend ; 


libname content json fileref=response ;

data _null_ ;
  set content.message_function_call ;
  call execute(cats('%function_call(name=',name, ',arguments=%nrbquote(', arguments, '))')) ;
run ; 